# Tutorial02 - Add transform keys

## Overview

🚀 By the last tutorial, you have learned the concept of ``Actor``, ``Level`` and ``Sequence``. In this tutorial, you will learn how to add transform keys to an ``Actor`` in order to render multiple poses of it. By the end of this tutorial, you will be able to:
- Open Blender/Unreal Engine by XRFeitoria
- Import meshes as the ``Actor``s in the ``Level``
- Set ``Actor``s' scale in the ``Level``
- Create a `Sequence` for rendering and adding transform keys to `Actor`s
- Add a camera in the `Sequence`
- Render and save images

## 1. Preparation

Similar as Tutorial 01, specify your engine path and initialize XRFeitoria.

In [ ]:
import xrfeitoria as xf

In [ ]:
# Replace with your executable path

# Blender
# engine_exec_path = 'D:/Program Files/Blender Foundation/Blender 3.3/blender.exe'

# Unreal Engine
engine_exec_path = 'D:/Program Files/Epic Games/UE_5.1/Engine/Binaries/Win64/UnrealEditor-Cmd.exe'

In [ ]:
from pathlib import Path

exec_path_stem = Path(engine_exec_path).stem.lower()
if 'blender' in exec_path_stem:
    # Open Blender
    xf_runner = xf.init_blender(exec_path=engine_exec_path, background=False, new_process=True)
elif 'unreal' in exec_path_stem:
    # Unreal Engine requires a project to be opened
    # Here we use a sample project, which is downloaded from the following link
    # You can also prepare your own project
    import shutil
    from xrfeitoria.utils.downloader import download
    unreal_project_zip = download(url='https://openxrlab-share.oss-cn-hongkong.aliyuncs.com/xrfeitoria/unreal_project/XRFeitoriaUnreal_Sample.zip', 
                                    dst_dir="./tutorial02/assets/")
    shutil.unpack_archive(filename=unreal_project_zip, extract_dir='./tutorial02/assets/')

    # Open Unreal Engine
    xf_runner = xf.init_unreal(exec_path=engine_exec_path, 
                                background=False, 
                                new_process=True, 
                                project_path='./tutorial02/assets/XRFeitoriaUnreal_Sample/XRFeitoriaUnreal_Sample.uproject')

✨ Now you can see a new Blender/Unreal Engine process has started.

## 3. Import meshes

Download the meshes of furnitures in [OmniObject3D](https://omniobject3d.github.io/) and the mesh of koupen chan, and import them as ``Actor``s in the ``Level``.

In [ ]:
import shutil
from xrfeitoria.utils.downloader import download

# Download the meshes
assets_zip = download('http://file.bj.zoe.sensetime.com/resources/meihaiyi/xrfeitoria/assets/tutorial02.zip', dst_dir="./tutorial02/assets/")
shutil.unpack_archive(filename=assets_zip, extract_dir='./tutorial02/assets/')

chair_path = Path("./tutorial02/assets/chair_001/Scan/chair.obj").resolve()
sofa_path = Path("./tutorial02/assets/sofa_001/Scan/sofa.obj").resolve()
table_path = Path("./tutorial02/assets/table_002/Scan/table.obj").resolve()
koupen_chan_path = Path("./tutorial02/assets/koupen_chan.fbx").resolve()

# Import the meshes
furnitures = {}
furnitures['chair'] = xf_runner.Actor.import_from_file(file_path=chair_path)
furnitures['table'] = xf_runner.Actor.import_from_file(file_path=table_path)
furnitures['sofa'] = xf_runner.Actor.import_from_file(file_path=sofa_path)
koupen_chan = xf_runner.Actor.import_from_file(file_path=koupen_chan_path)

Switch to the engine window, and you can see the meshes has been imported.

But the Actors are too large, so let's scale them down to a proper size.

Here we repeatly adjust the scale of the Actors until their size are below 2.0m.

In [ ]:
for _, furniture in furnitures.items():
    while max(furniture.dimensions) > 2.0:
        furniture.scale = tuple([s * 0.1 for s in furniture.scale])
        
while max(koupen_chan.dimensions) > 2.0:
    koupen_chan.scale = tuple([s * 0.1 for s in koupen_chan.scale])

Now they look like:

[image]

If you use ``Unreal Engine``, the final step before rendering is to save the ``Level``. You can save the ``Level`` by

In [ ]:
# save the level
if xf_runner.engine == 'unreal':
    xf_runner.utils.save_current_level()   

## 4. Add a sequence for rendering and adding transform keys

``Sequence`` is a multifunctional class in XRFeitoria. It can be used for:
- rendering
- adding transform keys
- grouping different objects.

Here, we use it for rendering and adding transform keys.

In [ ]:
import random
import copy
from xrfeitoria.data_structure.models import RenderPass
from xrfeitoria.data_structure.models import SequenceTransformKey as SeqTransKey

# Use `with` statement to create a sequence, and it will be automatically close the sequence after the code block is executed.
# The argument `seq_length` controls the number of frames to be rendered. 
with xf_runner.Sequence.new(seq_name="MySequence", seq_length=20) as seq:
    ##############################
    ##### Add transform keys #####
    ##############################

    # We want to add transform keys to the furnitures to randomly change their locations and rotations in the room.
    transform_keys = {}
    actor_idx = 0
    
    for name, actor in furnitures.items():
        actor_rotation = actor.rotation
        actor_scale = actor.scale
        for i in range(20):
            random_location = (random.uniform(actor_idx * 1.0, actor_idx * 1.0 + 1.0), 
                               random.uniform(0.0, 2.0), 
                               1.0)
            random_rotation = (actor_rotation[0],
                               actor_rotation[1],
                               actor_rotation[2] + random.randint(0, 3) * 90)

            transform_keys[name].append(
                SeqTransKey(
                    frame=i,
                    location=random_location,
                    rotation=random_rotation,
                    scale=actor_scale,
                    interpolation='AUTO',
                )            
            )
        # The function `use_actor_with_keys` sets transform keys for the actor in the sequence.
        # The transform keys are only stored in the sequence.
        # When the sequence is closed, the actor's location, rotation and scale will be restored to its original state(without transform keys).
        seq.use_actor_with_keys(actor=actor, transform_keys=transform_keys[name])
        actor_idx += 1
    
    # Add transform keys to koupen_chan to make it always sit on the table
    koupen_chan_transform_keys = copy.deepcopy(transform_keys['table'])
    for keys in koupen_chan_transform_keys:
        keys.location = (keys.location[0], keys.location[1], keys.location[2] + furnitures['table'].dimensions[2] * 0.5)
        keys.scale = koupen_chan.scale
    # The function `use_actor_with_keys` sets transform keys for the actor in the sequence.
    # The transform keys are only stored in the sequence.
    # When the sequence is closed, the actor will be restored to its original state(without transform keys).
    seq.use_actor_with_keys(actor=koupen_chan, transform_keys=koupen_chan_transform_keys)

    #####################
    ##### Rendering #####
    #####################

    # Add a camera and make it look at the specified location
    camera_location = (1.5, -2.0, 3.0)
    camera_rotation = xf_runner.utils.get_rotation_to_look_at(location=camera_location, target=(1.5, 1.0, 1.0))
    seq.spawn_camera(location=camera_location, rotation=camera_rotation, fov=90)

    # Add a render job to renderer
    # In render job, you can specify the output path, resolution, render passes, etc.
    # The ``output_path`` is the path to save the rendered data.
    # The ``resolution`` is the resolution of the rendered image.
    # The ``render_passes`` define what kind of data you want to render, such as img, depth, normal, etc.
    # and what kind of format you want to save, such as png, exr, etc.
    seq.add_to_renderer(
        output_path=f'./tutorial02/outputs/{xf_runner.engine}/',
        resolution=(1280, 720),
        render_passes=[RenderPass('img', 'png')],
    )

## 5. Render

The following code renders all the render jobs and save the images to the ``output_path`` set in ``seq.add_to_renderer`` above.

In [ ]:
xf_runner.render()

Check the ``output_path``, and you can see the rendered images like this:

[image]

## 6. Close the engine

Finally, close the engine by:

In [ ]:
xf_runner.close()

Ref to [api docs](../../apis/xrfeitoria.rst), you can always use ``with`` statement to ensure the engine is closed when the codes are finished.